In [10]:
"""
Script to find studies with activation in ROIs. 
- Loads the NiMARE Dataset from derivatives/neurosynth_terms_dataset.pkl.gz
- Identifies studies that have activation in 9 ROIs (by mask overlap)
- Outputs a CSV file with the selected study IDs and metadata to derivatives/macm/lCaud33_studies.csv
"""

import os
import os.path as op
import nibabel as nib
from nilearn.image import threshold_img
import numpy as np
import csv
from nimare.dataset import Dataset

In [11]:
def binarize(img, z_thresh=2.33, clust_thresh=10):
    
    data = np.asarray(img.dataobj)
    uniq = np.unique(data[np.isfinite(data)])
    if np.all(np.isin(uniq, [0, 1])) and uniq.size <= 2:
        return img
    thr = threshold_img(img, threshold=z_thresh, cluster_threshold=clust_thresh)
    data = (thr.get_fdata() > 0).astype(np.int16)
    return nib.Nifti1Image(data, thr.affine, thr.header)

In [ ]:
# Set up paths for notebook execution (no __file__)
PROJECT_DIR = os.getcwd()
DERIVATIVES_DIR = op.join(PROJECT_DIR, "derivatives")
DSET_PATH = op.join(DERIVATIVES_DIR, "neurosynth_terms_dataset.pkl.gz")
ROI_DIR = op.join(PROJECT_DIR, "dset", "ROIs")
OUT_ROOT = op.join(DERIVATIVES_DIR, "macm")

print(f"\tLoading dataset: {DSET_PATH}", flush=True)
dset = Dataset.load(DSET_PATH)

In [ ]:
roi_files = [f for f in os.listdir(ROI_DIR) if f.endswith('.nii') or f.endswith('.nii.gz')]
for roi_file in roi_files:
    roi_path = op.join(ROI_DIR, roi_file)
    roi_img = binarize(nib.load(roi_path))
    mask = roi_img.get_fdata().astype(bool)
    csv_path = op.join(roi_out_dir, f"{roi_stem}_studies.csv")
    if csv_path and os.path.exists(csv_path):
        print(f"\tCSV already exists at: {csv_path}.")
        
    else:
        print(f"\tFinding studies with activation in {roi_file}...", flush=True)
        study_ids = dset.ids
        selected_ids = []
        coords_df = dset.coordinates
        coords_by_id = {sid: group[["x", "y", "z"]].values for sid, group in coords_df.groupby("id")}
        for sid in study_ids:
            coords = coords_by_id.get(sid, np.empty((0, 3)))
            found = False
            for c in coords:
                ijk = np.round(np.linalg.inv(roi_img.affine) @ np.append(c[:3], 1)).astype(int)[:3]
                if (0 <= ijk[0] < mask.shape[0] and 0 <= ijk[1] < mask.shape[1] and 0 <= ijk[2] < mask.shape[2]):
                    if mask[tuple(ijk)]:
                        found = True
                        break
            if found:
                selected_ids.append(sid)
        print(f"\tFound {len(selected_ids)} studies with {roi_file}.", flush=True)
        if not selected_ids:
            print(f"\tNo studies found with {roi_file}. Skipping.", flush=True)
            continue

        # Output selected study IDs and metadata to CSV in respective folder
        roi_stem = roi_file.replace('.nii.gz', '').replace('.nii', '')
        roi_out_dir = op.join(OUT_ROOT, roi_stem)
        os.makedirs(roi_out_dir, exist_ok=True)

        with open(csv_path, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            header = ["id"]
            meta_fields = []
            if hasattr(dset, "metadata") and dset.metadata is not None and not dset.metadata.empty:
                meta_fields = list(sorted({k for md in dset.metadata.to_dict('records') for k in md}))
                header += meta_fields
            writer.writerow(header)
            for sid in selected_ids:
                row = [sid]
                if meta_fields:
                    md_row = dset.metadata.loc[dset.metadata['id'] == sid]
                    if not md_row.empty:
                        md = md_row.iloc[0].to_dict()
                        row += [md.get(f, "") for f in meta_fields]
                    else:
                        row += ["" for _ in meta_fields]
                writer.writerow(row)
        print(f"\tWrote selected study IDs and metadata to: {csv_path}", flush=True)
        print(f"\tSelected: {len(selected_ids)} studies written to CSV for {roi_file}.", flush=True)

	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv.
	CSV already exists at: /Users/chloehampson/Desktop/implicit-learning-macm/deriv

In [19]:
import pandas as pd
import os
import os.path as op

# Directory where the CSVs are saved
macm_dir = op.join(os.getcwd(), "derivatives", "macm")

# Find all ROI subfolders
roi_folders = [f for f in os.listdir(macm_dir) if op.isdir(op.join(macm_dir, f))]

# Load each CSV into a dictionary
roi_csvs = {}
for roi in roi_folders:
    csv_path = op.join(macm_dir, roi, f"{roi}_studies.csv")
    if op.exists(csv_path):
        df = pd.read_csv(csv_path)
        roi_csvs[roi] = df
        print(f"Loaded {csv_path} with {len(df)} rows.")
    else:
        print(f"CSV not found for ROI: {roi}")

print(roi_csvs)
# roi_csvs is a dictionary: key=ROI name, value=DataFrame

Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rCaud31/rCaud31_studies.csv with 546 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/lMDN/lMDN_studies.csv with 400 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/PFC/PFC_studies.csv with 1076 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rIFGtri1/rIFGtri1_studies.csv with 274 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rGPi/rGPi_studies.csv with 318 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/lIFGop7/lIFGop7_studies.csv with 173 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/lGPi/lGPi_studies.csv with 310 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/rMDN/rMDN_studies.csv with 483 rows.
Loaded /Users/chloehampson/Desktop/implicit-learning-macm/derivatives/macm/lCaud33/lC

In [29]:
def compare_rois_and_save_csv(roi_names, output_csv_path=None):
    """
    Given a list of ROI names (matching subfolder names in derivatives/macm),
    find the intersection of their study IDs and save as a new CSV.
    If output_csv_path is not provided, it will be named automatically as 'roi1-roi2-...-studies.csv'.
    """
    # Load DataFrames for the specified ROIs
    dfs = []
    for roi in roi_names:
        csv_path = op.join(macm_dir, roi, f"{roi}_studies.csv")
        if not op.exists(csv_path):
            print(f"CSV not found for ROI: {roi}")
            return
        df = pd.read_csv(csv_path)
        dfs.append(df)
    # Find intersection of IDs
    common_ids = set(dfs[0]['id'])
    for df in dfs[1:]:
        common_ids &= set(df['id'])
    print(f"Found {len(common_ids)} common study IDs for ROIs: {roi_names}")
    # Optionally, merge metadata from the first ROI
    if common_ids:
        merged = dfs[0][dfs[0]['id'].isin(common_ids)].copy()
        if output_csv_path is None:
            roi_part = '-'.join(roi_names)
            output_csv_path = f"derivatives/macm/{roi_part}-studies.csv"
        merged.to_csv(output_csv_path, index=False)
        print(f"Saved intersection CSV to {output_csv_path}")
    else:
        print("No common study IDs found.")

def compare_all_rois_and_save_csv():
    """
    Find the intersection of study IDs across all ROI CSVs and save as 9ROIs-studies.csv.
    """
    roi_names = [f for f in os.listdir(macm_dir) if op.isdir(op.join(macm_dir, f))]
    if not roi_names:
        print("No ROI folders found in macm_dir.")
        return
    compare_rois_and_save_csv(roi_names, output_csv_path="derivatives/macm/9ROIs-studies.csv")


In [30]:
compare_all_rois_and_save_csv()

Found 0 common study IDs for ROIs: ['rCaud31', 'lMDN', 'PFC', 'rIFGtri1', 'rGPi', 'lIFGop7', 'lGPi', 'rMDN', 'lCaud33']
No common study IDs found.


In [34]:
compare_rois_and_save_csv(['PFC', 'lMDN', 'rMDN','lGPi', 'rGPi', 'rCaud31', 'lCaud33'])


Found 0 common study IDs for ROIs: ['PFC', 'lMDN', 'rMDN', 'lGPi', 'rGPi', 'rCaud31', 'lCaud33']
No common study IDs found.


In [35]:
compare_rois_and_save_csv(['PFC', 'lMDN', 'rMDN','lGPi', 'rGPi'])

Found 1 common study IDs for ROIs: ['PFC', 'lMDN', 'rMDN', 'lGPi', 'rGPi']
Saved intersection CSV to derivatives/macm/PFC-lMDN-rMDN-lGPi-rGPi-studies.csv


In [36]:
compare_rois_and_save_csv(['PFC', 'rCaud31', 'lCaud33'])

Found 16 common study IDs for ROIs: ['PFC', 'rCaud31', 'lCaud33']
Saved intersection CSV to derivatives/macm/PFC-rCaud31-lCaud33-studies.csv


In [37]:
compare_rois_and_save_csv(['PFC', 'lIFGop7', 'rIFGtri1'])

Found 2 common study IDs for ROIs: ['PFC', 'lIFGop7', 'rIFGtri1']
Saved intersection CSV to derivatives/macm/PFC-lIFGop7-rIFGtri1-studies.csv
